In [1]:
import sys
import os


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
print(sys.path[-1])

/home/cotsios/dsit/2nd-semester/ml-in-comp-bio/Assignment-2


In [2]:
from src.preprocessing import fill_nans_with_median
import pandas as pd

In [3]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(BASE_DIR, "data")

In [4]:
validation_set_fraction = 0.1
seed = 42

In [5]:
# Load the df
df = pd.read_csv(os.path.join(DATA_DIR, "breast_cancer.csv"))

df = df.drop(columns=["id"])

for col in df.columns:
    if " " in col:
        df.rename(columns={col: col.replace(" ", "_")}, inplace=True)

# Separate rows with and without NaNs in two dfs
n = df.shape[0]
nan_mask = df.isna().any(axis=1)
df_with_nans = df[nan_mask]
df_without_nans = df[~nan_mask]

# Get the ratio of each class
mal = df["diagnosis"].tolist().count("M")
ben = df["diagnosis"].tolist().count("B")
mal_ratio = mal / (ben + mal)
ben_ratio = ben / (ben + mal)

# Get the number of rows needed for each class
n_mal = int(n * validation_set_fraction * mal_ratio)
n_ben = int(n * validation_set_fraction * ben_ratio)

# Create the holdout set
val_set = pd.concat([
    df_without_nans[df_without_nans["diagnosis"] == "M"].sample(n=n_mal, random_state=seed),
    df_without_nans[df_without_nans["diagnosis"] == "B"].sample(n=n_ben, random_state=seed)
])
val_set["diagnosis"] = val_set["diagnosis"].apply(lambda x: 1 if x == "M" else 0)

# Remove rows entered in the holdout set from the previous df
df_without_nans = df_without_nans.drop(val_set.index)

# Combine the remaining and cleanup
val_set.reset_index(inplace=True, drop=True)
df = pd.concat([df_with_nans, df_without_nans], ignore_index=True)
df = pd.concat(
    [
        df["diagnosis"].apply(lambda x: 1 if x == "M" else 0),
        fill_nans_with_median(
            df=df.drop(columns=["diagnosis"]),
            target=df["diagnosis"],
            class_based=True
        )
    ],
    axis=1
)

In [6]:
df

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,0,11.500,18.45,73.28,407.4,0.09345,0.05991,0.02638,0.02069,0.1834,...,12.970,22.46,83.12,508.9,0.11830,0.10490,0.08105,0.06544,0.2740,0.07699
1,0,12.950,16.02,83.14,513.7,0.10050,0.07943,0.06155,0.03370,0.1730,...,13.740,19.93,88.81,585.4,0.14830,0.16715,0.22410,0.10560,0.3380,0.09584
2,0,9.904,18.06,64.60,302.4,0.09699,0.12940,0.13070,0.03716,0.1669,...,11.260,24.39,73.07,546.7,0.13010,0.29500,0.34860,0.09910,0.2614,0.11620
3,1,11.420,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.34805,0.68690,0.25750,0.6638,0.17300
4,0,12.180,17.84,77.79,451.1,0.10450,0.07057,0.02490,0.02941,0.1714,...,12.830,20.92,82.14,495.2,0.11400,0.09358,0.04980,0.05882,0.2227,0.07376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,0,12.200,15.21,78.01,457.9,0.08673,0.06545,0.01994,0.01692,0.1638,...,13.750,21.38,91.11,583.1,0.12560,0.19280,0.11670,0.05556,0.2661,0.07961
457,0,9.720,18.22,60.73,288.1,0.06950,0.02344,0.00000,0.00000,0.1653,...,9.968,20.83,62.25,303.8,0.07117,0.02729,0.00000,0.00000,0.1909,0.06559
458,0,11.890,21.17,76.39,433.8,0.09773,0.08120,0.02555,0.02179,0.2019,...,13.050,27.21,85.09,522.9,0.14260,0.21870,0.11640,0.08263,0.3075,0.07351
459,0,13.000,25.13,82.61,520.2,0.08369,0.05073,0.01206,0.01762,0.1667,...,14.340,31.88,91.06,628.5,0.12180,0.10930,0.04462,0.05921,0.2306,0.06291


In [7]:
val_set

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,1,18.030,16.85,117.50,990.0,0.08947,0.12320,0.109000,0.062540,0.1720,...,20.380,22.02,133.30,1292.0,0.12630,0.26660,0.429000,0.153500,0.2842,0.08225
1,1,13.430,19.63,85.84,565.4,0.09048,0.06288,0.058580,0.034380,0.1598,...,17.980,29.87,116.60,993.6,0.14010,0.15460,0.264400,0.116000,0.2884,0.07371
2,1,15.490,19.97,102.40,744.7,0.11600,0.15620,0.189100,0.091130,0.1929,...,21.200,29.41,142.10,1359.0,0.16810,0.39130,0.555300,0.212100,0.3187,0.10190
3,1,20.310,27.06,132.90,1288.0,0.10000,0.10880,0.151900,0.093330,0.1814,...,24.330,39.16,162.30,1844.0,0.15220,0.29450,0.378800,0.169700,0.3151,0.07999
4,1,20.180,19.54,133.80,1250.0,0.11330,0.14890,0.213300,0.125900,0.1724,...,22.030,25.07,146.00,1479.0,0.16650,0.29420,0.530800,0.217300,0.3032,0.08075
5,1,16.130,20.68,108.10,798.8,0.11700,0.20220,0.172200,0.102800,0.2164,...,20.960,31.48,136.80,1315.0,0.17890,0.42330,0.478400,0.207300,0.3706,0.11420
6,1,17.540,19.32,115.10,951.6,0.08968,0.11980,0.103600,0.074880,0.1506,...,20.420,25.84,139.50,1239.0,0.13810,0.34200,0.350800,0.193900,0.2928,0.07867
7,1,14.580,21.53,97.41,644.8,0.10540,0.18680,0.142500,0.087830,0.2252,...,17.620,33.21,122.40,896.9,0.15250,0.66430,0.553900,0.270100,0.4264,0.12750
8,1,12.680,23.84,82.69,499.0,0.11220,0.12620,0.112800,0.068730,0.1905,...,17.090,33.47,111.80,888.3,0.18510,0.40610,0.402400,0.171600,0.3383,0.10310
9,1,14.860,23.21,100.40,671.4,0.10440,0.19800,0.169700,0.088780,0.1737,...,16.080,27.78,118.60,784.7,0.13160,0.46480,0.458900,0.172700,0.3000,0.08701


In [8]:
df.to_csv(
    os.path.join(DATA_DIR, "clean.csv"),
    index=False
)
val_set.to_csv(
    os.path.join(DATA_DIR, "holdout.csv"),
    index=False
)